In [14]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
mkdir: cannot create directory ‘rars’: File exists
copying adventure.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Adventure (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Air Raid (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Alien.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying crazy_climber.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Crazy Climber.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/crazy_climber.bin
copying 

In [15]:
!pip install --user keras-rl
!pip install --user keras==2.2.4
!pip install --user Pillow
!pip install --user h5py

In [16]:
!pip install  gym keras-rl2 gym[atari]

In [17]:
import numpy as np
import gym
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor


In [18]:
# Get the environment and extract the number of actions and the input shape.
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4
env = gym.make('Breakout-v0')
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n
print(nb_actions)
input_shape_ = env.observation_space.shape
env.unwrapped.get_action_meanings()

4


['NOOP', 'FIRE', 'RIGHT', 'LEFT']

In [19]:

WINDOW_LENGTH = 4

def build_model(input_shape, nb_actions):

  model = Sequential()

  model.add(Convolution2D(32, (8, 8), strides=(4, 4), input_shape=input_shape))
  model.add(Activation('relu'))
  model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
  model.add(Activation('relu'))
  model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
  model.add(Activation('relu'))
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dense(nb_actions))
  model.add(Activation('linear'))
  return model

model = build_model(input_shape=((WINDOW_LENGTH,) + input_shape_), nb_actions=nb_actions)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 4, 51, 39, 32)     6176      
_________________________________________________________________
activation_5 (Activation)    (None, 4, 51, 39, 32)     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 24, 18, 64)     32832     
_________________________________________________________________
activation_6 (Activation)    (None, 4, 24, 18, 64)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 22, 16, 64)     36928     
_________________________________________________________________
activation_7 (Activation)    (None, 4, 22, 16, 64)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 90112)            

In [20]:

def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=100000, window_length=4)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [21]:
dqn = build_agent(model, nb_actions)

def train():

  dqn.compile(Adam(lr=1e-3), metrics=['mae'])
  filename = 'policy.h5'
  dqn.fit(env,
          nb_steps=1000,
          log_interval=100, visualize=False, verbose=2)
  # After training is done, we save the final weights one more time.
  dqn.save_weights(filename, overwrite=True)

train()

Training for 1000 steps ...
 298/1000: episode: 1, duration: 11.273s, episode steps: 298, steps per second:  26, episode reward:  2.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 1.547 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
 474/1000: episode: 2, duration: 6.512s, episode steps: 176, steps per second:  27, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.460 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
 716/1000: episode: 3, duration: 8.907s, episode steps: 242, steps per second:  27, episode reward:  1.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 1.500 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
done, took 38.143 seconds


In [23]:
!pip install colabgymrender
from colabgymrender.recorder import Recorder

def play(env):
  dqn.load_weights('policy.h5')
  # dqn.test(env, nb_episodes=10, visualize=True)

  """recorde and visualization the game"""
  directory = './video'
  env = Recorder(env, directory)

  observation = env.reset()
  terminal = False
  while not terminal:
    action = env.action_space.sample()
    observation, reward, terminal, info = env.step(action)
  env.play()

play(env=env)